In [1]:
import sys
sys.path.append("../src")
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata
import utils
import pandas as pd

# Utils

In [ ]:
private_key, public_key = utils.generate_nostr_keypair()
e = utils.generate_event(private_key, public_key, 1, [], "test")
assert utils.verify_sig(e['id'], e['pubkey'], e['sig']), "Signature verification failed"
assert utils.calc_event_id(e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content']) == e['id'], "Event ID calculation failed"
e = Event(e['id'], e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content'], e['sig'])
e

# Database

In [2]:
bigbrotr = Bigbrotr(
    host="localhost",
    port=5431,
    user="admin",
    password="admin",
    dbname="bigbrotr"
)

In [3]:
bigbrotr.connect()

In [ ]:
query = "SELECT * FROM events"
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
df = {
    "id": [],
    "pubkey": [],
    "created_at": [],
    "kind": [],
    "tags": [],
    "content": [],
    "sig": []
}
for row in rows:
    df["id"].append(row[0])
    df["pubkey"].append(row[1])
    df["created_at"].append(row[2])
    df["kind"].append(row[3])
    df["tags"].append(row[4])
    df["content"].append(row[5])
    df["sig"].append(row[6])
df = pd.DataFrame(df)

In [ ]:
query = "SELECT * FROM relay_metadata"
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
for row in rows:
    print(row)

In [4]:
query = "SELECT COUNT(*) FROM events"
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
print(f"Number of events in the database: {rows[0][0]}")

Number of events in the database: 471502


In [5]:
bigbrotr.close()

# relay_urls

In [ ]:
import pandas as pd
relays_url = pd.read_csv("../seed/relays_url.csv")
relays_url = relays_url.groupby('relay_url').agg({'count': 'sum'}).reset_index()
relays_url = relays_url.sort_values(by='count', ascending=False).reset_index(drop=True)
relays_url.to_csv("../seed/relays_url.csv", index=False)